In [2]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.prompts import ChatPromptTemplate

In [7]:
def load_and_process_data():
    file_path = "../datos_universidad.txt"
    loader = TextLoader(file_path, encoding="utf-8")
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(docs)
    embeddings = OllamaEmbeddings(
        base_url="http://localhost:11434",
        model="nomic-embed-text"
    )
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    
    return vectorstore

In [8]:
vectorstore = load_and_process_data()

In [26]:
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vectorstore.similarity_search(last_query)

    context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    #print(context)
    system_message = (
        f"""Eres un asistente útil de la universidad.
        Usa los siguientes fragmentos de contexto recuperado para responder la pregunta.
        Mantén la respuesta concisa y en español.\n\n
        {context}"""
    )
    return system_message

llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2:3b", temperature=0)
agent = create_agent(llm, tools=[], middleware=[prompt_with_context])

In [28]:
query = "Hablame un poco sobre la UCLV"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hablame un poco sobre la UCLV
================================== Ai Message ==================================

La Universidad Central de las Villas (UCLV) es una institución educativa cubana ubicada en Santa Clara, Villa Clara. Fue fundada en 1960 y se caracteriza por su compromiso con la formación integral de sus estudiantes, enfocándose en la educación superior y la investigación.

La UCLV ha tenido un desarrollo significativo a lo largo de los años, destacando su participación activa en la vida académica y cultural nacional. En 2007, se sometió a un proceso de acreditación y obtuvo la categoría de Carrera Acreditada, demostrando su compromiso con la calidad educativa.

En 2013, se reacreditó y obtuvo la categoría de Carrera de Excelencia, lo que refleja sus logros en áreas como el desarrollo docente-educativo y la formación de egresados. La institución ha implementado planes de mejora continuos para s

In [30]:
response = agent.invoke({"messages":["Hi"]})

In [37]:
response["messages"][-1].content

'¡Hola! ¿En qué puedo ayudarte hoy? Estoy aquí para responder a tus preguntas y proporcionarte información útil sobre la Universidad Central Marta Abreu de Las Villas.'